In [ ]:
# -*- coding: utf-8 -*-
import json
import os
from tqdm import tqdm
import glob
import cv2
import numpy as np

In [ ]:
def convert_label_json(json_dir, save_dir, classes):
    json_paths = os.listdir(json_dir)
    classes = classes.split(',')

    for json_path in tqdm(json_paths):
        path = os.path.join(json_dir, json_path)
        with open(path, 'r') as load_f:
            json_dict = json.load(load_f)
        h, w = json_dict['imageHeight'], json_dict['imageWidth']

        # save txt path
        txt_path = os.path.join(save_dir, json_path.replace('json', 'txt'))
        txt_file = open(txt_path, 'w')

        for shape_dict in json_dict['shapes']:
            label = shape_dict['label']
            label_index = classes.index(label)
            points = shape_dict['points']

            points_nor_list = []

            for point in points:
                points_nor_list.append(point[0] / w)
                points_nor_list.append(point[1] / h)

            points_nor_list = list(map(lambda x: str(x), points_nor_list))
            points_nor_str = ' '.join(points_nor_list)

            label_str = str(label_index) + ' ' + points_nor_str + '\n'
            txt_file.writelines(label_str)

In [ ]:
# 直接设置参数
json_dir = 'C:/Users/admin/Desktop/CQU_EIE/dataEngineer/trafSignReco/dataset/json_labels'
save_dir = 'C:/Users/admin/Desktop/CQU_EIE/dataEngineer/trafSignReco/dataset/labels'
classes = 'red light,green light,yellow light,turn right,parking,zebra'

# 转换标签
convert_label_json(json_dir, save_dir, classes)

In [ ]:
def check_labels(txt_labels, images_dir):
    txt_files = glob.glob(txt_labels + "/*.txt")
    for txt_file in txt_files:
        filename = os.path.splitext(os.path.basename(txt_file))[0]
        pic_path = os.path.join(images_dir, filename + ".jpg")  # 确保路径正确

        img = cv2.imread(pic_path)
        if img is None:
            print(f"Image not found or unable to read: {pic_path}")
            continue  # 如果图片无法读取，跳过该文件

        height, width, _ = img.shape

        file_handle = open(txt_file)
        cnt_info = file_handle.readlines()
        new_cnt_info = [line_str.replace("\n", "").split(" ") for line_str in cnt_info]

        color_map = {"0": (0, 255, 255)}  # 用黄色绘制边框
        for new_info in new_cnt_info:
            print(new_info)
            s = []
            for i in range(1, len(new_info), 2):
                b = [float(tmp) for tmp in new_info[i:i + 2]]
                s.append([int(b[0] * width), int(b[1] * height)])

            # 设置线条厚度为2，调整为更粗的线条
            cv2.polylines(img, [np.array(s, np.int32)], True, color_map.get(new_info[0]), thickness=2)

        cv2.namedWindow('img2', 0)
        cv2.imshow('img2', img)
        cv2.waitKey()

In [ ]:
images_dir = 'C:/Users/admin/Desktop/CQU_EIE/dataEngineer/trafSignReco/dataset/images/'
check_labels(save_dir, images_dir)